In [102]:
"""CS579: Assignment 0
Collecting a political social network
In this assignment, I've given you a list of Twitter accounts of 4
U.S. presedential candidates.
The goal is to use the Twitter API to construct a social network of these
accounts. We will then use the [networkx](http://networkx.github.io/) library
to plot these links, as well as print some statistics of the resulting graph.
1. Create an account on [twitter.com](http://twitter.com).
2. Generate authentication tokens by following the instructions [here](https://dev.twitter.com/docs/auth/tokens-devtwittercom).
3. Add your tokens to the key/token variables below. (API Key == Consumer Key)
4. Be sure you've installed the Python modules
[networkx](http://networkx.github.io/) and
[TwitterAPI](https://github.com/geduldig/TwitterAPI). Assuming you've already
installed [pip](http://pip.readthedocs.org/en/latest/installing.html), you can
do this with `pip install networkx TwitterAPI`.
OK, now you're ready to start collecting some data!
I've provided a partial implementation below. Your job is to complete the
code where indicated.  You need to modify the 10 methods indicated by
#TODO.
Your output should match the sample provided in Log.txt.
"""

# Imports you'll need.
from collections import Counter
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from TwitterAPI import TwitterAPI

consumer_key = 'tb9zMiyUJ9UeX6blluI4Wlws8'
consumer_secret = 'XlGNdXHj6Rd2LJ2t9LodnCLydvO2kZ4KLoq95VnQRXfuKXDgZn'
access_token = '3231404389-56xTceJ255ar5Sj2sQtWuDj8z0grnNedlqOt5AC'
access_token_secret = 'qwPAoWezVnfgeQCCaJy03rOQUJG2Z9ZACzVPIjcqu7IKD'


In [103]:
# This method is done for you. Make sure to put your credentials in the file twitter.cfg.
def get_twitter():
    """ Construct an instance of TwitterAPI using the tokens you entered above.
    Returns:
      An instance of TwitterAPI.
      
    """
    consumer_key = 'tb9zMiyUJ9UeX6blluI4Wlws8'
    consumer_secret = 'XlGNdXHj6Rd2LJ2t9LodnCLydvO2kZ4KLoq95VnQRXfuKXDgZn'
    access_token = '3231404389-56xTceJ255ar5Sj2sQtWuDj8z0grnNedlqOt5AC'
    access_token_secret = 'qwPAoWezVnfgeQCCaJy03rOQUJG2Z9ZACzVPIjcqu7IKD'
    return TwitterAPI(consumer_key, consumer_secret, access_token, access_token_secret)


   



In [104]:
twi = get_twitter()

In [105]:
def read_screen_names(filename):
    
    
    """
    Read a text file containing Twitter screen_names, one per line.
    Params:
        filename....Name of the file to read.
    Returns:
        A list of strings, one per screen_name, in the order they are listed
        in the file.
    Here's a doctest to confirm your implementation is correct.
    >>> read_screen_names('candidates.txt')
    ['DrJillStein', 'GovGaryJohnson', 'HillaryClinton', 'realDonaldTrump']

    """
    with open(filename) as f:
        content = [a.strip('\n') for a in f.readlines()]
    return content

In [106]:
twitter = get_twitter()
screen_names = read_screen_names('candidates.txt')
print('Read screen names: %s' % screen_names)


Read screen names: ['DrJillStein', 'GovGaryJohnson', 'HillaryClinton', 'realDonaldTrump']


In [107]:
# I've provided the method below to handle Twitter's rate limiting.
# You should call this method whenever you need to access the Twitter API.
def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request; e.g., "friends/ids"
      params ..... A parameter dict for the request, e.g., to specify
                   parameters like screen_name or count.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request.status_code == 200:
            return request
        else:
            print('Got error %s \nsleeping for 15 minutes.' % request.text)
            sys.stderr.flush()
            time.sleep(61 * 15)


In [108]:
def get_users(twitter, screen_names):
    """Retrieve the Twitter user objects for each screen_name.
    Params:
        twitter........The TwitterAPI object.
        screen_names...A list of strings, one per screen_name
    Returns:
        A list of dicts, one per user, containing all the user information
        (e.g., screen_name, id, location, etc)
    See the API documentation here: https://dev.twitter.com/rest/reference/get/users/lookup
    In this example, I test retrieving two users: twitterapi and twitter.
    >>> twitter = get_twitter()
    >>> users = get_users(twitter, ['twitterapi', 'twitter'])
    >>> [u['id'] for u in users]
    [6253282, 783214]
    """
    users= []
    for user in screen_names:
        users.append([user_info for user_info in robust_request(twitter,'users/lookup',{'screen_name' : user})][0])
    return users

In [109]:
users=get_users(twitter, screen_names)


In [110]:
def get_friends(twitter, screen_name):
    """ Return a list of Twitter IDs for users that this person follows, up to 5000.
    See https://dev.twitter.com/rest/reference/get/friends/ids
    Note, because of rate limits, it's best to test this method for one candidate before trying
    on all candidates.
    Args:
        twitter.......The TwitterAPI object
        screen_name... a string of a Twitter screen name
    Returns:
        A list of ints, one per friend ID, sorted in ascending order.
    Note: If a user follows more than 5000 accounts, we will limit ourselves to
    the first 5000 accounts returned.
    In this test case, I return the first 5 accounts that I follow.
    >>> twitter = get_twitter()
    >>> get_friends(twitter, 'aronwc')[:5]
    [695023, 1697081, 8381682, 10204352, 11669522]
    """
    ###TODO
    return [friend_id for friend_id in robust_request(twitter,'friends/ids',{'screen_name' : screen_name, 'count' :5000})]

In [111]:
def add_all_friends(twitter, users):
    """ Get the list of accounts each user follows.
    I.e., call the get_friends method for all 4 candidates.
    Store the result in each user's dict using a new key called 'friends'.
    Args:
        twitter...The TwitterAPI object.
        users.....The list of user dicts.
    Returns:
        Nothing
    >>> twitter = get_twitter()
    >>> users = [{'screen_name': 'aronwc'}]
    >>> add_all_friends(twitter, users)
    >>> users[0]['friends'][:5]
    [695023, 1697081, 8381682, 10204352, 11669522]
    """
    ###TODO
    for user in users:
        user['friends'] = get_friends(twitter,user['screen_name'])
    pass

In [112]:
add_all_friends(twitter, users)


In [113]:
def print_num_friends(users):
    """Print the number of friends per candidate, sorted by candidate name.
    See Log.txt for an example.
    Args:
        users....The list of user dicts.
    Returns:
        Nothing
    """
    ###TODO
    for user in users:
        print (user['screen_name'],'\t', len(user['friends']) )
    pass

In [114]:
def count_friends(users):
    """ Count how often each friend is followed.
    Args:
        users: a list of user dicts
    Returns:
        a Counter object mapping each friend to the number of candidates who follow them.
        Counter documentation: https://docs.python.org/dev/library/collections.html#collections.Counter
    In this example, friend '2' is followed by three different users.
    >>> c = count_friends([{'friends': [1,2]}, {'friends': [2,3]}, {'friends': [2,3]}])
    >>> c.most_common()
    [(2, 3), (3, 2), (1, 1)]
    """
    friends = []
    for frnds in users:
        friends.extend(frnds['friends'])
    return Counter(friends)
    pass

In [115]:
def mutual(user1,user2):
    return set(user1['friends']) & set(user2['friends'])

In [116]:
def friend_overlap(users):
    """
    Compute the number of shared accounts followed by each pair of users.
    Args:
        users...The list of user dicts.
    Return: A list of tuples containing (user1, user2, N), where N is the
        number of accounts that both user1 and user2 follow.  This list should
        be sorted in descending order of N. Ties are broken first by user1's
        screen_name, then by user2's screen_name (sorted in ascending
        alphabetical order). See Python's builtin sorted method.
    In this example, users 'a' and 'c' follow the same 3 accounts:
    >>> friend_overlap([
    ...     {'screen_name': 'a', 'friends': ['1', '2', '3']},
    ...     {'screen_name': 'b', 'friends': ['2', '3', '4']},
    ...     {'screen_name': 'c', 'friends': ['1', '2', '3']},
    ...     ])
    [('a', 'c', 3), ('a', 'b', 2), ('b', 'c', 2)]
    """
    ###TODO
    overlap = []
    for i in range(len(users)):
        for j in range(i+1,len(users)):
            N = len(mutual(users[i],users[j]))
            overlap.append((users[i]['screen_name'],users[j]['screen_name'],N))
    return sorted(overlap, key=lambda x: x[2])[::-1]
    pass

In [117]:
def followed_by_hillary_and_donald(users, twitter):
    """
    Find and return the screen_name of the one Twitter user followed by both Hillary
    Clinton and Donald Trump. You will need to use the TwitterAPI to convert
    the Twitter ID to a screen_name. See:
    https://dev.twitter.com/rest/reference/get/users/lookup
    Params:
        users.....The list of user dicts
        twitter...The Twitter API object
    Returns:
        A string containing the single Twitter screen_name of the user
        that is followed by both Hillary Clinton and Donald Trump.
    """
    ###TODO
    mutual_h_t = list(mutual(users[2],users[3]))
    mutual_follow = []
    for i in mutual_h_t:
        mutual_follow.append([user_info for user_info in robust_request(twitter,'users/lookup',{'user_id' : i})][0])
        
    return mutual_follow[0]['name']
    pass

In [118]:
followed_by_hillary_and_donald(users, twitter)

'Earvin Magic Johnson'

In [119]:
print('Friend Overlap:\n%s' % str(friend_overlap(users)))

Friend Overlap:
[('DrJillStein', 'GovGaryJohnson', 349), ('GovGaryJohnson', 'realDonaldTrump', 12), ('DrJillStein', 'HillaryClinton', 10), ('GovGaryJohnson', 'HillaryClinton', 9), ('DrJillStein', 'realDonaldTrump', 8), ('HillaryClinton', 'realDonaldTrump', 1)]


In [120]:
def create_graph(users, friend_counts):
    """ Create a networkx undirected Graph, adding each candidate and friend
        as a node.  Note: while all candidates should be added to the graph,
        only add friends to the graph if they are followed by more than one
        candidate. (This is to reduce clutter.)
        Each candidate in the Graph will be represented by their screen_name,
        while each friend will be represented by their user id.
    Args:
      users...........The list of user dicts.
      friend_counts...The Counter dict mapping each friend to the number of candidates that follow them.
    Returns:
      A networkx Graph
    """
    ###TODO
    import matplotlib.pyplot as plt  
    %matplotlib inline
    
    graph = nx.DiGraph()
    a=[]        
    for user in users:
        a = user['friends']
        graph.add_node(user['name'])
        print ("asn")
        for id in a:
            print (id)
            graph.add_node(id)     
            #graph.add_edge(user['name'],id) 
            nx.draw(graph, with_labels=True)
    
    pass



In [121]:
def main():
    """ Main method. You should not modify this. """
    twitter = get_twitter()
    screen_names = read_screen_names('candidates.txt')
    print('Established Twitter connection.')
    print('Read screen names: %s' % screen_names)
    users = sorted(get_users(twitter, screen_names), key=lambda x: x['screen_name'])
    print('found %d users with screen_names %s' %
          (len(users), str([u['screen_name'] for u in users])))
    add_all_friends(twitter, users)
    print('Friends per candidate:')
    print_num_friends(users)
    friend_counts = count_friends(users)
    print('Most common friends:\n%s' % str(friend_counts.most_common(5)))
    print('Friend Overlap:\n%s' % str(friend_overlap(users)))
    print('User followed by Hillary and Donald: %s' % followed_by_hillary_and_donald(users, twitter))

  



In [122]:
if __name__ == '__main__':
    main()

Established Twitter connection.
Read screen names: ['DrJillStein', 'GovGaryJohnson', 'HillaryClinton', 'realDonaldTrump']
found 4 users with screen_names ['DrJillStein', 'GovGaryJohnson', 'HillaryClinton', 'realDonaldTrump']
Friends per candidate:
DrJillStein 	 4159
GovGaryJohnson 	 3124
HillaryClinton 	 748
realDonaldTrump 	 42
Most common friends:
[(15846407, 3), (19608297, 3), (41877508, 2), (12, 2), (130613344, 2)]
Friend Overlap:
[('DrJillStein', 'GovGaryJohnson', 349), ('GovGaryJohnson', 'realDonaldTrump', 12), ('DrJillStein', 'HillaryClinton', 10), ('GovGaryJohnson', 'HillaryClinton', 9), ('DrJillStein', 'realDonaldTrump', 8), ('HillaryClinton', 'realDonaldTrump', 1)]


TwitterConnectionError: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=5)

In [123]:

users

[{'contributors_enabled': False,
  'created_at': 'Thu Feb 04 04:55:27 +0000 2010',
  'default_profile': False,
  'default_profile_image': False,
  'description': 'Green Party candidate for President, activist, medical doctor, environmental health advocate.  Account managed by https://t.co/cJMqxQwvT8 staff. #ItsInOurHands',
  'entities': {'description': {'urls': [{'display_url': 'Jill2016.com',
      'expanded_url': 'http://Jill2016.com',
      'indices': [114, 137],
      'url': 'https://t.co/cJMqxQwvT8'}]},
   'url': {'urls': [{'display_url': 'Jill2016.com',
      'expanded_url': 'http://Jill2016.com',
      'indices': [0, 23],
      'url': 'https://t.co/cJMqxQwvT8'}]}},
  'favourites_count': 13837,
  'follow_request_sent': False,
  'followers_count': 219868,
  'following': False,
  'friends': [22948191,
   40716090,
   125465007,
   122130797,
   227720229,
   456794981,
   441389311,
   35175054,
   289019104,
   30576467,
   763849988988211200,
   18021496,
   774180818,
   1684206